In [2]:
import os
import seaborn as sns
path = '/Users/connormcdonald/Desktop/Masters/MIT807/Gartner Repository/Analysis/Figures'
import sys
sys.path.insert(1, '/Users/connormcdonald/Desktop/Masters/MIT807/Gartner Repository/Data Collection')
from configs import *
import numpy as np
import statsmodels.formula.api as smf
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt

# from mpl_toolkits.axisartist.axislines import SubplotZero
# from pylab import text
# matplotlib.use("pgf")
# matplotlib.rcParams.update({
#     "pgf.texsystem": "lualatex",
#     'font.family': 'serif',
#     'text.usetex': True,
#     'pgf.rcfonts': False,
# })

from sqlalchemy import create_engine

engine = create_engine('mysql+mysqlconnector://'+user+':'+passwd+'@'+ip+':3306/'+schema1)

In [3]:
df = pd.read_csv('/Users/connormcdonald/Desktop/Masters/MIT807/Data/twitterMAU.csv')
df['period']  = pd.to_datetime(df['period'])

In [4]:
df1 = df.iloc[:38,:]

In [8]:
import matplotlib.dates as mdates


degree = 3
y = np.array(df1['mau'].to_list())
X = np.array(mdates.date2num(df1['period']))
date_index = []
idx = 0
for i in X:
    date_index.append(idx)
    idx += 1

z = np.polyfit(date_index, y, degree)
f = np.poly1d(z)

df_trend = pd.DataFrame(columns=['X', 'y','d'])
df_trend['X'] = date_index
df_trend['y'] = y
df_trend['d'] = X


z = np.polyfit(df_trend.d, df_trend.y, degree)
model = np.poly1d(z)
results = smf.ols(formula='y ~ model(X)', data=df_trend).fit()

# print(results.summary().as_latex())

In [17]:
model

poly1d([ 5.01067557e-10, -6.04624158e-05,  1.67401913e+00, -1.31289063e+04])

In [16]:
z

array([ 5.01067557e-10, -6.04624158e-05,  1.67401913e+00, -1.31289063e+04])

In [14]:
X = mdates.date2num(df['period'])

In [19]:
x_fit = mdates.date2num(df['period'])
y_fit = [model(_x) for _x in x_fit]

In [25]:
df['period']

0    2010-03-31
1    2010-06-30
2    2010-09-30
3    2010-12-31
4    2011-03-31
5    2011-06-30
6    2011-09-30
7    2011-12-31
8    2012-03-31
9    2012-06-30
10   2012-09-30
11   2012-12-31
12   2013-03-31
13   2013-06-30
14   2013-09-30
15   2013-12-31
16   2014-03-31
17   2014-06-30
18   2014-09-30
19   2014-12-31
20   2015-03-31
21   2015-06-30
22   2015-09-30
23   2015-12-31
24   2016-03-31
25   2016-06-30
26   2016-09-30
27   2016-12-31
28   2017-03-31
29   2017-06-30
30   2017-09-30
31   2017-12-31
32   2018-03-31
33   2018-06-30
34   2018-09-30
35   2018-12-31
36   2019-03-31
37   2020-03-31
38   2020-06-30
39   2020-09-30
40   2020-12-31
41   2021-03-31
42   2021-06-30
43   2021-09-30
44   2021-12-31
Name: period, dtype: datetime64[ns]